In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import keras_tuner as kt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
train_dataset_path = "C:/Users/Hemanth Kumar Reddy/Documents/NNDL/Image2/archive (2)/asl_alphabet_train/asl_alphabet_train"
test_dataset_path = "C:/Users/Hemanth Kumar Reddy/Documents/NNDL/Image2/archive (2)/asl_alphabet_test/asl_alphabet_test"


In [3]:
img_size = (64, 64)
batch_size = 32

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dataset_path,
    image_size=img_size,
    batch_size=batch_size,
    seed=42,
    validation_split=0.2,
    subset="training"
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dataset_path,
    image_size=img_size,
    batch_size=batch_size,
    seed=42,
    validation_split=0.2,
    subset="validation"
)

Found 86994 files belonging to 29 classes.
Using 69596 files for training.
Found 86994 files belonging to 29 classes.
Using 17398 files for validation.


In [5]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Classes: {class_names}")


Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [6]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


In [7]:
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

In [8]:
def build_hybrid_model(hp):
    model = models.Sequential()
    
    for i in range(hp.Int("conv_layers", 2, 3)):  
        model.add(layers.Conv2D(
            filters=hp.Choice(f"filters_{i}", [32, 64, 128]),
            kernel_size=(3, 3),
            activation='relu'
        ))
        model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())

    for i in range(hp.Int("dense_layers", 1, 2)):  
        model.add(layers.Dense(
            units=hp.Choice(f"dense_units_{i}", [256, 512, 1024]),
            activation='relu'
        ))
        model.add(layers.Dropout(hp.Float("dropout", 0.2, 0.5, step=0.1)))  # Tune dropout
    
    model.add(layers.Dense(num_classes, activation='softmax'))

    learning_rate = hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [9]:
tuner = kt.RandomSearch(
    build_hybrid_model,
    objective="val_accuracy",
    max_trials=5, 
    executions_per_trial=1,
    directory="tuner_results",
    project_name="asl_sign_language"
)

In [13]:
tuner.search(train_ds, validation_data=val_ds, epochs=5, callbacks=[EarlyStopping(monitor='val_loss', patience=2)])

Trial 5 Complete [00h 20m 30s]
val_accuracy: 0.9922404885292053

Best val_accuracy So Far: 0.9922404885292053
Total elapsed time: 01h 51m 17s


In [14]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters found:", best_hps.values)

Best hyperparameters found: {'conv_layers': 2, 'filters_0': 64, 'filters_1': 128, 'dense_layers': 2, 'dense_units_0': 256, 'dropout': 0.2, 'learning_rate': 0.001, 'filters_2': 64, 'dense_units_1': 256}
